# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [ ]:
%matplotlib inline

In [3]:
# Import libraries
import numpy as np
import pandas as pd
import csv
import time
from sklearn import cross_validation
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split, KFold
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import f1_score

# First, decide how many training vs test samples you want
from sklearn.cross_validation import train_test_split

import pylab as pl
#import matplotlib.pyplot as pl
from sklearn.preprocessing import scale

In [4]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [5]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = student_data.shape[0]
n_features = student_data.shape[1] -1
n_passed =  student_data[student_data['passed'] =='yes'].shape[0]
n_failed =  student_data[student_data['passed'] == 'no'].shape[0]
grad_rate = (n_passed*1.0) / (n_students*1.0) * 100
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(float(grad_rate))

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [6]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
#print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed


### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [7]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Analysis of Data

In [8]:
#Format Target yes/no values with 1/0
y = pd.DataFrame(y_all, index = y_all.index)
y = y.replace(['yes', 'no'], [1, 0])
#in the form (X, 1), but the method expects a 1d array and has to be in the form (X, )

#print student_data.mean(numeric_only = True)
#print student_data.std(numeric_only = True) 
#print student_data.cov()
#box_plots = student_data.boxplot(widths = .5, return_type = 'axes')

#join dataset
df_data = pd.concat([X_all, y], axis = 1)
df1 = df_data.describe()
#print df_data
#print(df1.unstack())

#Data Correlation:
df = df_data.corr()
corr_target = df.ix[-1][:-1] #Target is last column in the DataFrame
predict = corr_target.sort_values(ascending=False)
df_sort = corr_target.sort_values(ascending=False)
#print df_sort

### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [9]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

y = student_data['passed']
#print y

def Stratified_Shuffle_Split(X,y,num_test):
    sss = StratifiedShuffleSplit(y, 1, test_size=num_test, random_state = None)
    for train_index, test_index in sss:
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    return X_train, X_test, y_train, y_test
    
X_train, X_test, y_train, y_test = Stratified_Shuffle_Split(X_all, y, num_test)
print "Training Set: {0:.2f} Samples".format(X_train.shape[0])
print "Testing Set: {0:.2f} Samples".format(X_test.shape[0])

Training Set: 300.00 Samples
Testing Set: 95.00 Samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What is the theoretical O(n) time & space complexity in terms of input size?
- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [10]:
# Train a model
def train_classifier(clf, X_train, y_train):
    print "Training {}:".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    train_clf_time = end - start
    print "Training Time (secs): {:.3f}".format(train_clf_time)
    return train_clf_time

In [11]:
# Predict on training set and compute F1 score
def predict_labels(clf, features, target):
    print "Predicting labels using {}:".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    prediction_time = end - start
    print "Prediction Time (secs): {:.3f}".format(prediction_time)
    return (f1_score(target.values, y_pred, pos_label='yes'), prediction_time)

In [12]:
# Train and Predict
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    print "Testing set size: {}".format(len(X_test))
    train_diff = train_classifier(clf, X_train, y_train)
    
    # Train and predict on diff. training set sizes
    f1_score_train, pred_time_train = predict_labels(clf, X_train, y_train)
    print "F1 score for training set:",(f1_score_train)
    # Predict on test data
    f1_score_test, pred_time_test = predict_labels(clf, X_test, y_test)
    print "F1 score for test set:",(f1_score_test)
    
    return (f1_score_train, f1_score_test, train_diff, pred_time_test)


### Model 1: Naive Bayes 

In [13]:
clf_NB = GaussianNB()

# Fit model to training data
train_classifier(clf_NB, X_train, y_train)

# Predict on training set and compute F1 score
train_f1_score = predict_labels(clf_NB, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score[0])

#Predict on Test Data
print "**********************************************************"
print "Testing Data Size:",len(X_train)
print "Testing Data Size:",len(X_test)
print "F1 score for test set:",(predict_labels(clf_NB, X_test, y_test))



Training GaussianNB:
Training Time (secs): 0.004
Predicting labels using GaussianNB:
Prediction Time (secs): 0.001
F1 score for training set: 0.810551558753
**********************************************************
Testing Data Size: 300
Testing Data Size: 95
F1 score for test set: Predicting labels using GaussianNB:
Prediction Time (secs): 0.003
(0.74626865671641784, 0.0031380653381347656)


### Model 2: Random Forest

In [14]:
clf_RF = RandomForestClassifier(n_estimators=10)

# Fit model to training data
train_classifier(clf_RF, X_train, y_train)

# Predict on training set and compute F1 score
train_f1_score = predict_labels(clf_RF, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

#Predict on Test Data
print "**********************************************************"
print "Training Data Size:",len(X_train)
print "Testing Data Size:",len(X_test)
print "F1 score for test set: {}".format(predict_labels(clf_RF, X_test, y_test))

Training RandomForestClassifier:
Training Time (secs): 0.065
Predicting labels using RandomForestClassifier:
Prediction Time (secs): 0.002
F1 score for training set: (0.98994974874371855, 0.0021262168884277344)
**********************************************************
Training Data Size: 300
Testing Data Size: 95
Predicting labels using RandomForestClassifier:
Prediction Time (secs): 0.002
F1 score for test set: (0.72307692307692306, 0.0016579627990722656)


### Model 3: Support Vector Machines

In [15]:
clf_SVC = SVC()

# Fit model to training data
train_classifier(clf_SVC, X_train, y_train)

# Fit model to training data
train_classifier(clf_SVC, X_train, y_train)

# Predict on training set and compute F1 score
train_f1_score = predict_labels(clf_SVC, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

#Predict on Test Data
print "**********************************************************"
print "Training Data Size:",len(X_train)
print "Testing Data Size:",len(X_test)
print "F1 score for test set: {}".format(predict_labels(clf_SVC, X_test, y_test))

Training SVC:
Training Time (secs): 0.061
Training SVC:
Training Time (secs): 0.014
Predicting labels using SVC:
Prediction Time (secs): 0.008
F1 score for training set: (0.87032967032967035, 0.00801396369934082)
**********************************************************
Training Data Size: 300
Testing Data Size: 95
Predicting labels using SVC:
Prediction Time (secs): 0.003
F1 score for test set: (0.80519480519480513, 0.0026900768280029297)


### Run All Models ( 10 iterations) 

In [16]:
def run_all_models(classifiers):
    for clf in classifiers:
        df = pd.DataFrame(columns = [
                    'Training_Size',
                    'Testing_Size',
                    'Training_Time',
                    'Prediction_Time',
                    'F1_Training_Score',
                    'F1_Testing_Score'])

        y = student_data['passed']
        X_train, X_test, y_train, y_test = Stratified_Shuffle_Split(X_all, y, num_test)
        
        num_times_to_run = 10
        sizes = [100,200,300]
        for size in sizes: 
            for x in range(0, num_times_to_run): 
                f1_score_train, f1_score_test, train_time, pred_time_test = train_predict(clf, X_train[:size], y_train[:size], X_test, y_test)
                        
                df = df.append({
                        'Training_Size': len(X_train[:size]),
                        'Testing_Size': X_test.shape[0],
                        'Training_Time': train_time,
                        'Prediction_Time': pred_time_test,
                        'F1_Training_Score': f1_score_train,
                        'F1_Testing_Score': f1_score_test}, 
                        ignore_index= True)
            
            df = df[(df.Training_Size == size)]
            df_mean = df.mean()
        
            print "**********************************************************"
            print "Mean Statistics:"
            print df_mean
            print "**********************************************************"

### Naive Bayes: 

In [17]:
run_all_models([clf_NB])

------------------------------------------
Training set size: 100
Testing set size: 95
Training GaussianNB:
Training Time (secs): 0.001
Predicting labels using GaussianNB:
Prediction Time (secs): 0.001
F1 score for training set: 0.579439252336
Predicting labels using GaussianNB:
Prediction Time (secs): 0.001
F1 score for test set: 0.466666666667
------------------------------------------
Training set size: 100
Testing set size: 95
Training GaussianNB:
Training Time (secs): 0.003
Predicting labels using GaussianNB:
Prediction Time (secs): 0.001
F1 score for training set: 0.579439252336
Predicting labels using GaussianNB:
Prediction Time (secs): 0.001
F1 score for test set: 0.466666666667
------------------------------------------
Training set size: 100
Testing set size: 95
Training GaussianNB:
Training Time (secs): 0.001
Predicting labels using GaussianNB:
Prediction Time (secs): 0.001
F1 score for training set: 0.579439252336
Predicting labels using GaussianNB:
Prediction Time (secs): 

### Random Forest: 

In [18]:
run_all_models([clf_RF])

------------------------------------------
Training set size: 100
Testing set size: 95
Training RandomForestClassifier:
Training Time (secs): 0.055
Predicting labels using RandomForestClassifier:
Prediction Time (secs): 0.002
F1 score for training set: 0.993006993007
Predicting labels using RandomForestClassifier:
Prediction Time (secs): 0.002
F1 score for test set: 0.739130434783
------------------------------------------
Training set size: 100
Testing set size: 95
Training RandomForestClassifier:
Training Time (secs): 0.042
Predicting labels using RandomForestClassifier:
Prediction Time (secs): 0.003
F1 score for training set: 0.992907801418
Predicting labels using RandomForestClassifier:
Prediction Time (secs): 0.002
F1 score for test set: 0.794520547945
------------------------------------------
Training set size: 100
Testing set size: 95
Training RandomForestClassifier:
Training Time (secs): 0.042
Predicting labels using RandomForestClassifier:
Prediction Time (secs): 0.002
F1 sco

### Support Vector Machines: 

------------------------------------------
Training set size: 100
Testing set size: 95
Training SVC:
Training Time (secs): 0.003
Predicting labels using SVC:
Prediction Time (secs): 0.001
F1 score for training set: 0.867924528302
Predicting labels using SVC:
Prediction Time (secs): 0.001
F1 score for test set: 0.792207792208
------------------------------------------
Training set size: 100
Testing set size: 95
Training SVC:
Training Time (secs): 0.002
Predicting labels using SVC:
Prediction Time (secs): 0.002
F1 score for training set: 0.867924528302
Predicting labels using SVC:
Prediction Time (secs): 0.001
F1 score for test set: 0.792207792208
------------------------------------------
Training set size: 100
Testing set size: 95
Training SVC:
Training Time (secs): 0.002
Predicting labels using SVC:
Prediction Time (secs): 0.002
F1 score for training set: 0.867924528302
Predicting labels using SVC:
Prediction Time (secs): 0.001
F1 score for test set: 0.792207792208
-------------------

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [20]:
# TODO: Fine-tune your model and report the best F1 score

# Reformat "Y" from 'yes/no' to '1/0'
def reformat(col_data):
    return col_data.replace(['yes', 'no'], [1, 0])

def iterate_fit_predict(number_runs):
        f1_scores = []
        gamma = []
        C = []

        y = reformat(student_data['passed'])
        
        for num in range(0,number_runs):
            X_train, X_test, y_train, y_test = Stratified_Shuffle_Split(X_all, y, num_test)
            clf_SVC = SVC()
            parameters = [{'C':[1,10,50,100,200,300,400,500,1000],
                         'gamma':[1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1],
                         'kernel': ['rbf']}]
    
            
            clf = GridSearchCV(clf_SVC, parameters, scoring = 'f1')
            
            # Fit the learner to the training data to obtain the best parameter set
            clf.fit(X_train, y_train)
            f1_scores.append(clf.score(X_test, y_test))
            gamma.append(clf.best_params_['gamma'])
            C.append(clf.best_params_['C'])
            clf = clf.best_estimator_
            #print clf
        
        df_f1 = pd.Series(f1_scores)
        df_gamma = pd.Series(gamma)
        df_C = pd.Series(C)
        
        print clf
        print "\nF1 Scores:"
        print df_f1
        print "\nC:"
        print df_C
        print "\nGamma:"
        print df_gamma

        print "\nAverage F1 Test Scores:"
        print df_f1.mean()
        print "\nAverage C:"
        print df_C.mean()
        print "\nAverage Gamma:"
        print df_gamma.mean()
        

In [224]:
iterate_fit_predict(10)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

F1 Scores:
0    0.818792
1    0.802632
2    0.797386
3    0.807692
4    0.828571
5    0.825175
6    0.815789
7    0.826667
8    0.813333
9    0.810127
dtype: float64

C:
0     10
1     10
2    200
3      1
4    200
5    500
6      1
7    200
8    200
9      1
dtype: int64

Gamma:
0    0.0010
1    0.0010
2    0.0001
3    0.1000
4    0.0001
5    0.0001
6    0.1000
7    0.0001
8    0.0001
9    0.1000
dtype: float64

Average F1 Test Scores:
0.814616376357

Average C:
132.3

Average Gamma:
0.03025
